In [1]:
!pip3 install seaborn tensorflow pandas tabulate shap

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import logging
from tqdm import tqdm
import time
import atexit
from tabulate import tabulate
import sys
import contextlib
from io import StringIO
from joblib import Parallel, delayed
import cloudpickle
import pickle
import gc

# Enable mixed precision training
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

# Configuration for Hyperparameter Set 1
CONFIG = {
    'input_csv': '../../data/daily_stock_price/sp500_top25_technical_indicators.csv',
    'output_csv': 'transformer_evaluation_results_walk_forward.csv',
    'target': 'Close',
    'look_back': 60,
    'forecast_horizon': 1,
    'batch_size': 64,
    'epochs': 40,
    'walkforward_retrain_step': 25,
    'train_start': '2001-01-01',
    'train_end': '2020-12-31',
    'test_start': '2021-01-01',
    'hyperparam_set': 5
}

FEATURE_COLUMNS = [
    'Close', 'SMA_20', 'RSI_14', 'MACD', 'MACD_Signal', 'MACD_Hist',
    'BB_Upper', 'BB_Lower', 'ATR_14', 'OBV',
    'Close_Lag_1', 'Close_Lag_2', 'Close_Lag_3', 'Close_Lag_5',
    'Volume_Lag_1', 'Volume_Lag_3', 'Daily_Return', 'Volatility_20',
    'High_Low_Range', 'Open_Close_Range', 'MACD_Hist_Slope'
]

# Custom stream for logging TensorFlow output
class LogStream:
    def __init__(self, logger, level=logging.INFO):
        self.logger = logger
        self.level = level
        self.buffer = StringIO()

    def write(self, message):
        self.buffer.write(message)
        if '\n' in message:
            self.flush()

    def flush(self):
        message = self.buffer.getvalue().rstrip()
        if message:
            self.logger.log(self.level, message)
        self.buffer.seek(0)
        self.buffer.truncate()

# Custom stream for tqdm to allow cell output
class TqdmStream:
    def write(self, message):
        sys.__stdout__.write(message)

    def flush(self):
        sys.__stdout__.flush()

# Create log file with timestamp
log_time = datetime.now().strftime("%Y%m%d_%H%M%S")
os.makedirs("log", exist_ok=True)
log_filename = os.path.join("log", f"transformer_log_set_{CONFIG['hyperparam_set']}_{log_time}.txt")
file_handler = logging.FileHandler(log_filename, mode='w')
stream_handler = logging.StreamHandler()
stream_handler.setLevel(logging.ERROR)  # Only show errors in cell output
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(processName)s - %(levelname)s - %(message)s',
    handlers=[file_handler, stream_handler]
)
logger = logging.getLogger(__name__)
atexit.register(lambda: [h.flush() for h in logger.handlers if hasattr(h, 'flush')])

# Custom callback for detailed epoch logging with loss collection
class EpochLogger(tf.keras.callbacks.Callback):
    def __init__(self, stock, hyperparam_set, walkforward_step=None):
        super(EpochLogger, self).__init__()
        self.stock = stock
        self.hyperparam_set = hyperparam_set
        self.walkforward_step = walkforward_step
        self.context = f"walk-forward step {walkforward_step}" if walkforward_step is not None else "initial training"
        self.epoch_losses = []

    def on_epoch_begin(self, epoch, logs=None):
        logger.info(f"Starting epoch {epoch + 1} for {self.stock} (Hyperparameter Set {self.hyperparam_set}, {self.context})")

    def on_epoch_end(self, epoch, logs=None):
        loss = logs.get('loss', 'N/A')
        self.epoch_losses.append({'epoch': epoch + 1, 'loss': loss})
        logger.info(f"Finished epoch {epoch + 1} for {self.stock} (Hyperparameter Set {self.hyperparam_set}, {self.context}) - Loss: {loss:.6f}")

def build_transformer_with_attention(look_back, n_features):
    inputs = tf.keras.Input(shape=(look_back, n_features))
    attn_output = tf.keras.layers.MultiHeadAttention(num_heads=2, key_dim=16)(inputs, inputs)
    x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(attn_output + inputs)
    x = tf.keras.layers.Dense(32, activation='relu')(x)
    x = tf.keras.layers.GlobalAveragePooling1D()(x)
    x = tf.keras.layers.Dense(32, activation='relu')(x)
    prediction = tf.keras.layers.Dense(1, dtype='float32')(x)
    model = tf.keras.Model(inputs=inputs, outputs=prediction)
    model.compile(optimizer='adam', loss='mse')
    return model

def create_sequences(data, look_back, forecast_horizon, target_index=0):
    X, y = [], []
    for i in range(len(data) - look_back - forecast_horizon + 1):
        X.append(data[i:i+look_back, :])
        y.append(data[i + look_back:i + look_back + forecast_horizon, target_index])
    return np.array(X, dtype=np.float32), np.array(y, dtype=np.float32)

def process_stock(stock, df, hyperparam_set, saliency_dir, config):
    os.makedirs(saliency_dir, exist_ok=True)
    start_time = time.time()
    retrain_count = 0
    epoch_losses = []

    try:
        logger.info(f"Processing {stock} for hyperparameter set {hyperparam_set}...")
        stock_df = df[df['symbol'] == stock].copy()
        stock_df['date'] = pd.to_datetime(stock_df['date'])
        stock_df = stock_df[(stock_df['date'] >= config['train_start'])]
        stock_df.sort_values('date', inplace=True)
        stock_df.set_index('date', inplace=True)

        features = stock_df[FEATURE_COLUMNS].dropna()
        if features.empty:
            logger.warning(f"{stock}: No data after dropping NaNs, skipping.")
            return None

        scaler = MinMaxScaler()
        features_scaled = scaler.fit_transform(features)
        target_index = FEATURE_COLUMNS.index(config['target'])
        X, y = create_sequences(features_scaled, config['look_back'], config['forecast_horizon'], target_index)

        dates = stock_df.index[config['look_back'] + config['forecast_horizon'] - 1:]
        date_mask = (dates >= pd.to_datetime(config['test_start']))
        if not any(date_mask):
            logger.warning(f"{stock}: No test data after {config['test_start']}, skipping.")
            return None

        split_idx = np.where(date_mask)[0][0]
        X_train_full, y_train_full = X[:split_idx], y[:split_idx]
        X_test, y_test = X[split_idx:], y[split_idx:]

        train_dataset = tf.data.Dataset.from_tensor_slices((X_train_full, y_train_full))
        train_dataset = train_dataset.cache().shuffle(1000).batch(config['batch_size']).prefetch(tf.data.AUTOTUNE)

        early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)
        epoch_logger = EpochLogger(stock, hyperparam_set)
        model = build_transformer_with_attention(config['look_back'], X.shape[2])

        @tf.function
        def train_step(x, y):
            with tf.GradientTape() as tape:
                predictions = model(x, training=True)
                loss = model.loss(y, predictions)
            gradients = tape.gradient(loss, model.trainable_variables)
            model.optimizer.apply_gradients(zip(gradients, model.trainable_variables))
            return loss

        log_stream = LogStream(logger)
        with contextlib.redirect_stdout(log_stream):
            model.fit(train_dataset, epochs=config['epochs'], callbacks=[early_stop, epoch_logger], verbose=1)
        epoch_losses.extend([{'ticker': stock, **loss} for loss in epoch_logger.epoch_losses])

        predictions = []
        history_X, history_y = X_train_full.tolist(), y_train_full.tolist()

        for i in tqdm(range(len(X_test)), desc=f"{stock}: Walk-forward steps", file=TqdmStream()):
            if i % config['walkforward_retrain_step'] == 0 and i > 0:
                logger.info(f"Starting retraining for {stock} at walk-forward step {i} (Hyperparameter Set {hyperparam_set})")
                model = build_transformer_with_attention(config['look_back'], X.shape[2])
                epoch_logger = EpochLogger(stock, hyperparam_set, walkforward_step=i)
                train_dataset = tf.data.Dataset.from_tensor_slices((np.array(history_X, dtype=np.float32), np.array(history_y, dtype=np.float32)))
                train_dataset = train_dataset.cache().shuffle(1000).batch(config['batch_size']).prefetch(tf.data.AUTOTUNE)
                with contextlib.redirect_stdout(log_stream):
                    model.fit(train_dataset, epochs=config['epochs'], callbacks=[early_stop, epoch_logger], verbose=1)
                epoch_losses.extend([{'ticker': stock, **loss} for loss in epoch_logger.epoch_losses])
                retrain_count += 1
                logger.info(f"Completed retraining for {stock} at walk-forward step {i} (Hyperparameter Set {hyperparam_set})")

            pred = model.predict(X_test[i:i+1], verbose=0, batch_size=1)
            predictions.append(pred[0][0])
            history_X.append(X_test[i].tolist())
            history_y.append([y_test[i][0]])

        preds_2d = np.tile(np.array(predictions)[:, np.newaxis], (1, X.shape[2]))
        y_test_flat = y_test[:, 0]
        y_true_2d = np.tile(y_test_flat[:, np.newaxis], (1, X.shape[2]))

        preds_inv = scaler.inverse_transform(preds_2d)[:, target_index]
        y_true_inv = scaler.inverse_transform(y_true_2d)[:, target_index]

        mse = mean_squared_error(y_true_inv, preds_inv)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_true_inv, preds_inv)
        r2 = r2_score(y_true_inv, preds_inv)

        logger.info(f"{stock} - MSE: {mse:.4f}, RMSE: {rmse:.4f}, MAE: {mae:.4f}, R2: {r2:.4f}")
        logger.info(f"{stock}: Done in {time.time() - start_time:.1f}s")
        logger.info(f"{stock}: Completed {config['epochs']} initial epochs and {retrain_count} retraining steps (Hyperparameter Set {hyperparam_set})")

        saliency = None
        try:
            input_sample = X_test[-1]
            input_tensor = tf.convert_to_tensor(input_sample[np.newaxis, ...], dtype=tf.float32)
            with tf.GradientTape() as tape:
                tape.watch(input_tensor)
                prediction = model(input_tensor)
            grads = tape.gradient(prediction, input_tensor).numpy()[0]
            saliency = np.mean(np.abs(grads), axis=0)

            plt.figure(figsize=(12, 6))
            plt.barh(FEATURE_COLUMNS, saliency)
            plt.xlabel("Average Absolute Gradient")
            plt.title(f"Saliency Map - {stock} (Set {hyperparam_set})")
            plt.tight_layout()
            plt.savefig(os.path.join(saliency_dir, f"saliency_{stock}_set_{hyperparam_set}.png"))
            plt.close()
            logger.info(f"Generated saliency map image for {stock} (Hyperparameter Set {hyperparam_set})")

        except Exception as grad_err:
            logger.warning(f"Saliency map failed for {stock}: {grad_err}")

        # Clear model and memory
        del model
        tf.keras.backend.clear_session()
        gc.collect()

        return {
            'Stock': stock,
            'MSE': mse,
            'RMSE': rmse,
            'MAE': mae,
            'R2': r2,
            'Saliency': saliency,
            'Epoch_Losses': epoch_losses,
            'batch_size': config['batch_size'],
            'epochs': config['epochs'],
            'walkforward_retrain_step': config['walkforward_retrain_step'],
            'hyperparam_set': config['hyperparam_set']
        }

    except Exception as e:
        logger.error(f"Error processing {stock}: {e}")
        return None

def generate_combined_saliency_plot(saliency_data, hyperparam_set, saliency_dir):
    try:
        if not saliency_data:
            logger.warning(f"No saliency data for hyperparameter set {hyperparam_set}")
            return
        logger.info(f"Generating combined saliency plot for hyperparameter set {hyperparam_set}")
        avg_saliency = np.mean(np.array(saliency_data), axis=0)
        
        plt.figure(figsize=(12, 6))
        plt.barh(FEATURE_COLUMNS, avg_saliency)
        plt.xlabel("Average Absolute Gradient")
        plt.title(f"Combined Saliency Map - All Stocks (Set {hyperparam_set})")
        plt.tight_layout()
        plt.savefig(os.path.join(saliency_dir, f"saliency_combined_set_{hyperparam_set}.png"))
        plt.close()
        logger.info(f"Generated combined saliency map image for all stocks (Hyperparameter Set {hyperparam_set})")

        saliency_df = pd.DataFrame([avg_saliency], columns=FEATURE_COLUMNS)
        saliency_csv_path = os.path.join(saliency_dir, f"saliency_combined_set_{hyperparam_set}.csv")
        saliency_df.to_csv(saliency_csv_path, index=False)
        logger.info(f"Saved combined saliency map CSV to {saliency_csv_path} (Hyperparameter Set {hyperparam_set})")

    except Exception as e:
        logger.error(f"Failed to generate combined saliency plot for hyperparameter set {hyperparam_set}: {e}")

def train_and_evaluate_transformer():
    try:
        logger.info("Loading input CSV")
        df = pd.read_csv(CONFIG['input_csv'])
        all_results = []

        stocks = df['symbol'].unique()
        logger.info(f"Processing {len(stocks)} stocks")

        os.makedirs("plot", exist_ok=True)
        os.makedirs("saliency_outputs", exist_ok=True)

        num_workers = 8

        saliency_dir = os.path.join("saliency_outputs", f"transformer_set_{CONFIG['hyperparam_set']}")
        try:
            os.makedirs(saliency_dir, exist_ok=True)
            logger.info(f"Created saliency directory: {saliency_dir}")
        except Exception as e:
            logger.error(f"Failed to create saliency directory {saliency_dir}: {e}")
            return None

        logger.info(f"Starting parallel processing for {len(stocks)} stocks with {num_workers} workers (Hyperparameter Set {CONFIG['hyperparam_set']})")
        try:
            results = Parallel(n_jobs=num_workers, backend='loky', verbose=1)(
                delayed(process_stock)(stock, df, CONFIG['hyperparam_set'], saliency_dir, CONFIG) for stock in stocks
            )
            logger.info(f"Completed parallel processing for {len(stocks)} stocks (Hyperparameter Set {CONFIG['hyperparam_set']})")
        except Exception as e:
            logger.error(f"Parallel processing failed for hyperparameter set {CONFIG['hyperparam_set']}: {e}")
            return None

        logger.info(f"Processing results for hyperparameter set {CONFIG['hyperparam_set']}")
        saliency_records = []
        epoch_loss_records = []
        metrics_records = []
        for result in results:
            if result:
                metrics_records.append({
                    'Stock': result['Stock'],
                    'MSE': result['MSE'],
                    'RMSE': result['RMSE'],
                    'MAE': result['MAE'],
                    'R2': result['R2'],
                    'batch_size': result['batch_size'],
                    'epochs': result['epochs'],
                    'walkforward_retrain_step': result['walkforward_retrain_step'],
                    'hyperparam_set': result['hyperparam_set']
                })
                if result['Saliency'] is not None:
                    saliency_records.append({
                        'ticker': result['Stock'],
                        **dict(zip(FEATURE_COLUMNS, result['Saliency']))
                    })
                epoch_loss_records.extend(result['Epoch_Losses'])

        logger.info(f"Saving saliency data to CSV for hyperparameter set {CONFIG['hyperparam_set']}")
        try:
            if saliency_records:
                saliency_df = pd.DataFrame(saliency_records)
                saliency_csv_path = os.path.join(saliency_dir, f"transformer_set_{CONFIG['hyperparam_set']}_saliency.csv")
                saliency_df.to_csv(saliency_csv_path, index=False)
                logger.info(f"Saved saliency data CSV to {saliency_csv_path} (Hyperparameter Set {CONFIG['hyperparam_set']})")
            else:
                logger.warning(f"No saliency data to save for hyperparameter set {CONFIG['hyperparam_set']}")
        except Exception as e:
            logger.error(f"Failed to save saliency data CSV for hyperparameter set {CONFIG['hyperparam_set']}: {e}")

        generate_combined_saliency_plot([r['Saliency'] for r in results if r and r['Saliency'] is not None], CONFIG['hyperparam_set'], saliency_dir)

        logger.info(f"Saving metrics to CSV for hyperparameter set {CONFIG['hyperparam_set']}")
        try:
            metrics_df = pd.DataFrame(metrics_records)
            metrics_csv_path = os.path.join(saliency_dir, f"transformer_set_{CONFIG['hyperparam_set']}_metrics.csv")
            metrics_df.to_csv(metrics_csv_path, index=False)
            logger.info(f"Saved metrics CSV to {metrics_csv_path} (Hyperparameter Set {CONFIG['hyperparam_set']})")
        except Exception as e:
            logger.error(f"Failed to save metrics CSV for hyperparameter set {CONFIG['hyperparam_set']}: {e}")

        logger.info(f"Saving epoch metrics to CSV for hyperparameter set {CONFIG['hyperparam_set']}")
        try:
            epoch_metrics_df = pd.DataFrame(epoch_loss_records)
            epoch_metrics_csv_path = os.path.join(saliency_dir, f"transformer_set_{CONFIG['hyperparam_set']}_epoch_metrics.csv")
            epoch_metrics_df.to_csv(epoch_metrics_csv_path, index=False)
            logger.info(f"Saved epoch metrics CSV to {epoch_metrics_csv_path} (Hyperparameter Set {CONFIG['hyperparam_set']})")
        except Exception as e:
            logger.error(f"Failed to save epoch metrics CSV for hyperparameter set {CONFIG['hyperparam_set']}: {e}")

        logger.info(f"Saving summary to CSV for hyperparameter set {CONFIG['hyperparam_set']}")
        try:
            summary = metrics_df[['MSE', 'RMSE', 'MAE', 'R2']].mean().to_frame().T
            summary['hyperparam_set'] = CONFIG['hyperparam_set']
            summary['batch_size'] = CONFIG['batch_size']
            summary['epochs'] = CONFIG['epochs']
            summary['walkforward_retrain_step'] = CONFIG['walkforward_retrain_step']
            summary_csv_path = os.path.join(saliency_dir, f"transformer_set_{CONFIG['hyperparam_set']}_summary.csv")
            summary.to_csv(summary_csv_path, index=False)
            logger.info(f"Saved summary CSV to {summary_csv_path} (Hyperparameter Set {CONFIG['hyperparam_set']})")
            logger.info("Performance Summary:")
            logger.info("\n" + tabulate(summary, headers='keys', tablefmt='github', showindex=False))
        except Exception as e:
            logger.error(f"Failed to save summary CSV for hyperparameter set {CONFIG['hyperparam_set']}: {e}")
            return None

        logger.info(f"Clearing memory after hyperparameter set {CONFIG['hyperparam_set']}")
        tf.keras.backend.clear_session()
        gc.collect()

        return metrics_df

    except Exception as e:
        logger.error(f"Training failed: {e}")
        return None

logger.info("Starting Transformer model training and evaluation...")
try:
    result_df = train_and_evaluate_transformer()
    if result_df is not None:
        logger.info("Evaluation complete.")
        logger.info("\n" + result_df.to_string())
except Exception as e:
    logger.error(f"Training failed: {e}")
    raise

/Users/kwklun/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
/Users/kwklun/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/kwklun/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/kwklun/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urlli

JPM: Walk-forward steps:  69%|######9   | 750/1083 [18:49<01:15,  4.40it/s]s]

/Users/kwklun/Library/Python/3.9/lib/python/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
2025-05-01 03:53:36.079684: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


TSLA: Walk-forward steps:  96%|#########6| 1025/1064 [20:07<00:10,  3.89it/s]

/Users/kwklun/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
2025-05-01 03:53:39.795034: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Max
2025-05-01 03:53:39.795055: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 48.00 GB
2025-05-01 03:53:39.795060: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 18.00 GB
2025-05-01 03:53:39.795075: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-05-01 03:53:39.795085: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with

...

2025-05-01 04:12:27.323580: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Max
2025-05-01 04:12:27.323601: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 48.00 GB
2025-05-01 04:12:27.323606: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 18.00 GB
2025-05-01 04:12:27.323623: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-05-01 04:12:27.323635: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


MA: Walk-forward steps:  78%|#######7  | 829/1064 [18:11<07:12,  1.84s/it]

2025-05-01 04:12:28.019154: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


BAC: Walk-forward steps: 100%|##########| 1083/1083 [19:41<00:00,  1.09s/it]


[Parallel(n_jobs=8)]: Done  25 out of  25 | elapsed: 100.5min finished
